## 파이썬 세션 3.2.1 : 리밸런싱 하기

# Periodic weight rebalancing

##  simple rtn x weight를 이용한 방법

In [30]:
# 먼저 Pandas 설치를 하고 불러옵니다.

!pip install pandas

In [31]:
import pandas as pd

In [32]:
import numpy as np

In [37]:
# 리밸런싱에 필요한 여러 함수들을 정의해 줍니다.


# 단순 수익률을 정의하는 함수입니다.
def get_returns_df(df, N=1, log=False):
    if log:
        return np.log(df / df.shift(N)).iloc[N-1:].fillna(0)
    else:
        return df.pct_change(N, fill_method=None).iloc[N-1:].fillna(0)

# 누적 수익률을 정의하는 함수입니다.
def get_cum_returns_df(return_df, log=False):
    if log:
        return np.exp(return_df.cumsum())
    else:
        return (1 + return_df).cumprod()    # same with (return_df.cumsum() + 1)



In [38]:
# 가상의 데이터 프레임을 만들어 줍니다. 
# 실제 데이터로 적용할 수 있으나 직관적인 이해를 위해 가상의 데이터를 만들었습니다.

df = pd.DataFrame(
    {
        "주식": [10, 15, 12, 13, 10, 11, 12],
        "채권": [10, 10, 8, 13, 12, 12, 12],
        "달러": [10, 12, 14, 16, 14, 14, 16],
    },
    index=pd.to_datetime(["2022-06-29", "2022-07-02", "2022-07-03", "2022-07-5", "2022-08-08", "2022-08-11", "2022-09-17"])
)
df

,주식,채권,달러
2022-06-29,10,10,10
2022-07-02,15,10,12
2022-07-03,12,8,14
2022-07-05,13,13,16
2022-08-08,10,12,14
2022-08-11,11,12,14
2022-09-17,12,12,16


In [39]:
# 변화율을 구하기 위해 새로운 변수를 지정해줍니다. 

rtn_df = get_returns_df(df, log=False)
shifted_rtn_df = rtn_df.shift(-1).fillna(0)   
shifted_rtn_df

,주식,채권,달러
2022-06-29,0.500000,0.000000,0.200000
2022-07-02,-0.200000,-0.200000,0.166667
2022-07-03,0.083333,0.625000,0.142857
2022-07-05,-0.230769,-0.076923,-0.125000
2022-08-08,0.100000,0.000000,0.000000
2022-08-11,0.090909,0.000000,0.142857
2022-09-17,0.000000,0.000000,0.000000


In [40]:
# 월 말에 정기 리밸런싱을 진행하기 위해
# drop_duplicates함수를 통해 데이터 내에서 월 말의 데이터를 가져옵니다. 

df['year'] = df.index.year
df['month'] = df.index.month

rebal_index = df.drop_duplicates(['year','month'], keep="last").index
df.drop(['year', 'month'], axis=1, inplace=True)

rebal_index

DatetimeIndex(['2022-06-29', '2022-07-05', '2022-08-11', '2022-09-17'], dtype='datetime64[ns]', freq=None)

In [41]:
# 월 초를 기준으로 하여 (1로 설정)
# 월 말까지의 가격 변화를 리스트로 저장하는 변수입니다.

month_cum_rtn_df_list = []
for start, end in zip(rebal_index[:-1], rebal_index[1:]):
    month_price_df = df.loc[start:end]
    month_cum_rtn_df = month_price_df / month_price_df.iloc[0]
    month_cum_rtn_df_list.append(month_cum_rtn_df)

In [42]:
# 제대로 작동하였는지 출력해 봅니다. 

month_cum_rtn_df_list[0]

,주식,채권,달러
2022-06-29,1.0,1.0,1.0
2022-07-02,1.5,1.0,1.2
2022-07-03,1.2,0.8,1.4
2022-07-05,1.3,1.3,1.6


In [43]:
# concat 함수를 이용하여 데이터를 이어줍니다.
# 이때 겹치는 날짜의 데이터들이 존재함을 알 수 있습니다.
# 이는 월 말이 다음달 데이터의 기준이 되고 또 해당 데이터의 마지막 날이 또 다음 달 시작의 기준이 되기 때문입니다. 

monthly_asset_flow_df = pd.concat(month_cum_rtn_df_list)
monthly_asset_flow_df

,주식,채권,달러
2022-06-29,1.000000,1.000000,1.000000
2022-07-02,1.500000,1.000000,1.200000
2022-07-03,1.200000,0.800000,1.400000
2022-07-05,1.300000,1.300000,1.600000
2022-07-05,1.000000,1.000000,1.000000
2022-08-08,0.769231,0.923077,0.875000
2022-08-11,0.846154,0.923077,0.875000
2022-08-11,1.000000,1.000000,1.000000
2022-09-17,1.090909,1.000000,1.142857


In [44]:
# .duplicated(keep=) 을 통해 중복되는 데이터를 뽑아줍니다.
# 이때 last를 통해 1 / 1 / 1 데이터가 있는 row만 남겨줍니다. 
# 그 이유는 리밸런싱 되어 다시 1 / 1 / 1로 설정된 것에만 관심이 있기 때문입니다.
# 또한 "~"monthly_asset에 주목해 봅시다. ~ (물결표시)는 역연산을 해주는 기능을 합니다. 

monthly_asset_flow_df = monthly_asset_flow_df.loc[~monthly_asset_flow_df.index.duplicated(keep="last")]
monthly_asset_flow_df

,주식,채권,달러
2022-06-29,1.000000,1.000000,1.000000
2022-07-02,1.500000,1.000000,1.200000
2022-07-03,1.200000,0.800000,1.400000
2022-07-05,1.000000,1.000000,1.000000
2022-08-08,0.769231,0.923077,0.875000
2022-08-11,1.000000,1.000000,1.000000
2022-09-17,1.090909,1.000000,1.142857


In [45]:
# 이제 해당 데이터 프레임을 내가 원래 설정한 포트폴리오 비중을 곱해줍니다. 
# 처음에 1로 설정한 이유는 나중에 그 비중을 직관적으로 알기 위함입니다.
# 이때 일자별 row의 합이 1이 넘을때도, 1이 되지 않을 때도 있습니다. 이는 자산 가격 변화에 따른 것입니다.
# 1보다 크다면 수익, 1보다 작다면 손해입니다.

monthly_asset_flow_df = monthly_asset_flow_df * [0.3, 0.5, 0.2]
monthly_asset_flow_df

,주식,채권,달러
2022-06-29,0.300000,0.500000,0.200000
2022-07-02,0.450000,0.500000,0.240000
2022-07-03,0.360000,0.400000,0.280000
2022-07-05,0.300000,0.500000,0.200000
2022-08-08,0.230769,0.461538,0.175000
2022-08-11,0.300000,0.500000,0.200000
2022-09-17,0.327273,0.500000,0.228571


In [46]:
# 자산 비중의 흐름을 알기 위해 각 row를 1이 되게 만들어줘야 합니다. (합쳐서 100%)
# 따라서 새로운 데이터프레임을 만들어줍니다. 
# 이를 통해 각 날짜별 row에서 비율의 합이 1이 됨을 알 수 있습니다.

weight_df = monthly_asset_flow_df.divide(monthly_asset_flow_df.sum(axis=1), axis=0)
weight_df

,주식,채권,달러
2022-06-29,0.300000,0.500000,0.200000
2022-07-02,0.378151,0.420168,0.201681
2022-07-03,0.346154,0.384615,0.269231
2022-07-05,0.300000,0.500000,0.200000
2022-08-08,0.266075,0.532151,0.201774
2022-08-11,0.300000,0.500000,0.200000
2022-09-17,0.309963,0.473555,0.216482


In [47]:
# 그 후 자산 가격의 변화율과 각 자산의 비중을 곱해주어
# 총 수익률을 구하는 데이터 프레임을 설정해줍니다.

net_rtn_df = shifted_rtn_df * weight_df
net_rtn_df

,주식,채권,달러
2022-06-29,0.150000,0.000000,0.040000
2022-07-02,-0.075630,-0.084034,0.033613
2022-07-03,0.028846,0.240385,0.038462
2022-07-05,-0.069231,-0.038462,-0.025000
2022-08-08,0.026608,0.000000,0.000000
2022-08-11,0.027273,0.000000,0.028571
2022-09-17,0.000000,0.000000,0.000000


In [48]:
# 그 후 시리즈의 형태로 일자별 수익률의 변화를 구합니다.

rtn_series = net_rtn_df.sum(axis=1).shift(1).fillna(0)
rtn_series

2022-06-29    0.000000
2022-07-02    0.190000
2022-07-03   -0.126050
2022-07-05    0.307692
2022-08-08   -0.132692
2022-08-11    0.026608
2022-09-17    0.055844
dtype: float64

In [49]:
# 마지막으로 1을 더해주면서 시작대비 포트폴리오의 변화를 구합니다.

(rtn_series + 1).cumprod() 

2022-06-29    1.000000
2022-07-02    1.190000
2022-07-03    1.040000
2022-07-05    1.360000
2022-08-08    1.179538
2022-08-11    1.210923
2022-09-17    1.278546
dtype: float64